In [ ]:
import random
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Dense, Input, Flatten, Concatenate, Dropout
from keras.models import Model, load_model, Sequential
from keras.utils import to_categorical, plot_model
import pydot
import keras.optimizers as kop
from keras import backend as K
import json
from nltk.corpus import movie_reviews
from NNmodels import OntoEmbeding2, Relation

In [2]:
MODEL_NAME = "IMDB_ONTO_EMBEDING2"

In [3]:
ttt = json.load(open(MODEL_NAME+'_data.json'))

In [4]:
entities = ttt['entities']
vocab = ttt['vocabulary']
relations = ttt['relations']

In [5]:
del ttt

In [6]:
cv = CountVectorizer(vocabulary=vocab)

In [10]:
modelonto = load_model(MODEL_NAME+'.model',custom_objects={'Relation':Relation})

In [11]:
# parámetros de la red
sentence_size = len(vocab)
neurons_per_ent = 10
neurons_per_rel = 2*neurons_per_ent
# construir la red ontológica
# entrada
sentence_input = Input(shape=(sentence_size,), name='input')
# red
onto = OntoEmbeding2(entities,relations)(sentence_input,neurons_per_ent,neurons_per_rel)
# modelo final
prmodel = Model(inputs=sentence_input, outputs=onto,name='OntologyEmbeding')

In [12]:
#copiar pesos de las capas entrenadas a la nueva red
for i in prmodel.layers:
    if i.weights and i.name:
        try:
            tt = modelonto.get_layer(i.name)
        except ValueError:
            continue
        i.set_weights(tt.get_weights())

In [13]:
prmodel.trainable = False

In [14]:
model = Sequential()
model.add(prmodel)
ll = len(entities)*neurons_per_ent + len(relations)*neurons_per_rel
model.add(Dense(ll,activation = 'relu'))
model.add(Dense(8*ll//10,activation = 'relu'))
model.add(Dense(8*ll//10,activation = 'relu'))
model.add(Dense(1, activation = 'sigmoid', name='out'))
#opt = kop.SGD(lr=0.5,momentum=0.9,decay=0.9,nesterov=True)
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['acc'])

In [15]:
del modelonto

In [16]:
MODEL_NAME +='_problem'
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

mm=model_to_dot(model, rankdir='LR').create(prog='dot', format='pdf')
with open(MODEL_NAME+'.pdf','wb') as f:
    f.write(mm)

In [17]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
OntologyEmbeding (Model)     (None, 240)               5350620   
_________________________________________________________________
dense_1 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_2 (Dense)              (None, 192)               46272     
_________________________________________________________________
dense_3 (Dense)              (None, 192)               37056     
_________________________________________________________________
out (Dense)                  (None, 1)                 193       
Total params: 5,491,981
Trainable params: 141,361
Non-trainable params: 5,350,620
_________________________________________________________________


In [18]:
#xtrain,xtest,ytrain,ytest = train_test_split(data, res, train_size=0.9)
#del data,res

In [19]:
import pickle

In [20]:
xtrain=pickle.load(open('xtrain.pickle','rb'))
ytrain=pickle.load(open('ytrain.pickle','rb'))
xtest=pickle.load(open('xtest.pickle','rb'))
ytest=pickle.load(open('ytest.pickle','rb'))

In [21]:
ltrain=len(xtrain)
ltest = len(xtest)
def generate(batch_size=32, train = False):
    while True: 
        data = []
        res = []
        if train:
            samples_n = np.random.randint(ltrain,size=batch_size)
            for i in samples_n:
                data.append(xtrain[i])
                res.append(ytrain[i])
        else:
            samples_n = np.random.randint(ltest,size=batch_size)
            for i in samples_n:
                data.append(xtest[i])
                res.append(ytest[i])
        data = cv.transform(data)
        res = np.array(res)
        yield data.toarray(),res

In [22]:
model.fit_generator(generate(100,True), validation_data=generate(100), validation_steps=10, epochs=10, steps_per_epoch=100)

Epoch 1/10
100/100 [==============================] - 14s 136ms/step - loss: 0.7861 - acc: 0.5275 - val_loss: 0.7010 - val_acc: 0.5160
Epoch 2/10
100/100 [==============================] - 14s 138ms/step - loss: 0.6984 - acc: 0.5109 - val_loss: 0.6895 - val_acc: 0.5520
Epoch 3/10
100/100 [==============================] - 14s 144ms/step - loss: 0.6942 - acc: 0.5326 - val_loss: 0.6881 - val_acc: 0.5260
Epoch 4/10
100/100 [==============================] - 14s 138ms/step - loss: 0.6933 - acc: 0.5305 - val_loss: 0.6911 - val_acc: 0.5120
Epoch 5/10
100/100 [==============================] - 14s 138ms/step - loss: 0.6916 - acc: 0.5350 - val_loss: 0.6886 - val_acc: 0.5360
Epoch 6/10
100/100 [==============================] - 14s 138ms/step - loss: 0.6918 - acc: 0.5350 - val_loss: 0.6923 - val_acc: 0.5240
Epoch 7/10
100/100 [==============================] - 14s 140ms/step - loss: 0.6924 - acc: 0.5248 - val_loss: 0.6887 - val_acc: 0.5270
Epoch 8/10
100/100 [==============================] - 1

In [23]:
model.save(MODEL_NAME+'.model')

In [24]:
model.layers[0].trainable=True
model.compile(optimizer='RMSprop', loss='binary_crossentropy', metrics=['acc'])

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
OntologyEmbeding (Model)     (None, 240)               5350620   
_________________________________________________________________
dense_1 (Dense)              (None, 240)               57840     
_________________________________________________________________
dense_2 (Dense)              (None, 192)               46272     
_________________________________________________________________
dense_3 (Dense)              (None, 192)               37056     
_________________________________________________________________
out (Dense)                  (None, 1)                 193       
Total params: 5,491,981
Trainable params: 5,491,981
Non-trainable params: 0
_________________________________________________________________


In [26]:
model.fit_generator(generate(100,True), validation_data=generate(100), validation_steps=10, epochs=50, steps_per_epoch=100)

Epoch 1/50
100/100 [==============================] - 32s 319ms/step - loss: 0.6905 - acc: 0.5383 - val_loss: 0.6722 - val_acc: 0.5800
Epoch 2/50
100/100 [==============================] - 30s 299ms/step - loss: 0.6664 - acc: 0.5931 - val_loss: 0.6555 - val_acc: 0.6050
Epoch 3/50
100/100 [==============================] - 30s 298ms/step - loss: 0.6330 - acc: 0.6377 - val_loss: 0.6239 - val_acc: 0.6480
Epoch 4/50
100/100 [==============================] - 32s 320ms/step - loss: 0.6070 - acc: 0.6654 - val_loss: 0.6269 - val_acc: 0.6450
Epoch 5/50
100/100 [==============================] - 29s 290ms/step - loss: 0.5868 - acc: 0.6836 - val_loss: 0.6613 - val_acc: 0.6070
Epoch 6/50
100/100 [==============================] - 29s 289ms/step - loss: 0.5895 - acc: 0.6820 - val_loss: 0.6239 - val_acc: 0.6590
Epoch 7/50
100/100 [==============================] - 31s 311ms/step - loss: 0.5719 - acc: 0.6992 - val_loss: 0.6314 - val_acc: 0.6480
Epoch 8/50
100/100 [==============================] - 3

In [27]:
model.save(MODEL_NAME+'.model')

In [28]:
loss, accuracy = model.evaluate_generator(generate(100),100)

In [29]:
print(f"Test loss: {loss:.3}")
print(f"Test accuracy: {accuracy:.3%}")

Test loss: 0.66
Test accuracy: 67.820%
